In [6]:
import numpy as np

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import vgg19
from tensorflow.keras import backend as K

In [ ]:
target_path = ''
style_path = ''

width, height = load_img(target_path).size
img_height = 400
img_width = int(width * img_height/height)

In [7]:
def preprocess_image(image_path):
    
    img = load_img(image_path, target_size = (image_height, image_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    
    return img

In [5]:
def deprocess_image(x):
    
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [ ]:
target_image = K.constant(preprocess_image(target_path))
style_ref_image = K.constant(preprocess_image(style_path))